In [1]:
import torch
from torch import nn
from torch.nn import functional as F

In [2]:
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
X = torch.rand(2, 20)
net(X)

tensor([[-0.0290,  0.2218,  0.0502,  0.1124,  0.0923,  0.0416, -0.0531,  0.0760,
          0.0937,  0.1511],
        [-0.0668,  0.1883,  0.1136, -0.0185,  0.0382,  0.0250,  0.0229,  0.0667,
          0.0487,  0.2592]], grad_fn=<AddmmBackward0>)

In [3]:
#net是一个对象，是nn.Module的子类，有__call__方法，所以可以当做函数调用
#net(X)相当于net.__call__(X)
#callable可以看某个对象是否可以被当做函数调用
callable(net)

True

In [4]:
class MPL(nn.Module):
    def __init__(self):
        #调用MPL的父类nn.Module的构造函数来执行必要的初始化
        #这样在类实例化时也可以指定其他函数参数，例如模型参数params
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    #定义模型的前向传播
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [5]:
net = MPL()
net(X)

tensor([[-0.1074,  0.0541, -0.0712,  0.0651,  0.0820,  0.1443, -0.0138,  0.1423,
          0.0105,  0.0869],
        [ 0.0422,  0.0738, -0.2088,  0.0511, -0.0014,  0.2440, -0.0857,  0.1450,
          0.0571,  0.0467]], grad_fn=<AddmmBackward0>)

In [33]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        #这里我用列表来储存层的，但是书上让用字典
        self.blocks = []
        for block in args:
            self.blocks.append(block)
        
    def forward(self, X):
        for block in self.blocks:
            X = block(X)
        return X

In [34]:
#用列表储存层是打印不出来net的各个层的
#应该是nn.Module中__str__()函数的原因
#也就是说，用字典储存层是为了配合nn.Module中的其他函数
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
print(net)

MySequential()


In [35]:
for param in net.parameters():
    print(param)

In [ ]:
y = net(X)
y.mean().backward()

In [17]:
blocks = []
blocks.append(nn.Linear(20, 256))
blocks.append(nn.ReLU())
blocks.append(nn.Linear(256, 10))
blocks, blocks[0].weight

([Linear(in_features=20, out_features=256, bias=True),
  ReLU(),
  Linear(in_features=256, out_features=10, bias=True)],
 Parameter containing:
 tensor([[-0.1624,  0.0558, -0.0061,  ..., -0.0964, -0.0113, -0.0331],
         [-0.2236,  0.1921,  0.0209,  ..., -0.1294,  0.1859,  0.1862],
         [ 0.1358, -0.0953, -0.0246,  ...,  0.0627,  0.0805,  0.0752],
         ...,
         [-0.1585,  0.1208,  0.1648,  ...,  0.1761,  0.0515,  0.0729],
         [ 0.1926, -0.1652, -0.1113,  ..., -0.1022,  0.0847, -0.0972],
         [-0.1218, -0.1662, -0.1272,  ...,  0.0881, -0.0791, -0.1411]],
        requires_grad=True))

In [22]:
result = X.clone()
for block in blocks:
    result = block(result)
result

tensor([[ 0.1750,  0.0244,  0.0492,  0.2723, -0.0485, -0.0486, -0.0341,  0.1803,
         -0.0604,  0.1274],
        [ 0.1393,  0.0010, -0.0430,  0.2210, -0.1947, -0.2531, -0.1204,  0.3382,
         -0.0273,  0.0317]], grad_fn=<AddmmBackward0>)

In [41]:
#书上是这样的：
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # 这里，module是Module子类的一个实例。我们把它保存在'Module'类的成员
            # 变量_modules中。_module的类型是OrderedDict
            self._modules[str(idx)] = module

    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X
'''_init__函数将每个模块逐个添加到有序字典_modules中。 
读者可能会好奇为什么每个Module都有一个_modules属性？ 
以及为什么我们使用它而不是自己定义一个Python列表？ 
简而言之，_modules的主要优点是： 
在模块的参数初始化过程中， 系统知道在_modules字典中查找需要初始化参数的子块。'''

'_init__函数将每个模块逐个添加到有序字典_modules中。 \n读者可能会好奇为什么每个Module都有一个_modules属性？ \n以及为什么我们使用它而不是自己定义一个Python列表？ \n简而言之，_modules的主要优点是： \n在模块的参数初始化过程中， 系统知道在_modules字典中查找需要初始化参数的子块。'

In [31]:
#用字典储存层就可以打印出来层
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
print(net)

MySequential(
  (0): Linear(in_features=20, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


In [43]:
net.state_dict()

OrderedDict([('linear.weight',
              tensor([[-0.0261, -0.0910, -0.0083,  0.0641, -0.0784,  0.1091, -0.0460, -0.2048,
                        0.2164, -0.1356, -0.1360,  0.0683, -0.0943, -0.1578, -0.0382,  0.1487,
                        0.2083, -0.2040,  0.0709, -0.0639],
                      [-0.0916, -0.2222, -0.1142,  0.1466,  0.0556,  0.1170, -0.2057, -0.2130,
                        0.1738, -0.2113,  0.0837,  0.1612, -0.0467,  0.0051, -0.0691,  0.1401,
                       -0.0967, -0.1489, -0.1231, -0.1285],
                      [ 0.0582,  0.2057, -0.1993,  0.2191, -0.0774,  0.1106,  0.2058,  0.1353,
                       -0.0669,  0.2225,  0.1395, -0.0626, -0.1654, -0.0642,  0.1524, -0.0398,
                       -0.0823,  0.0006, -0.1297, -0.0405],
                      [ 0.0352,  0.0861, -0.1645, -0.0623, -0.1900, -0.1139,  0.0946,  0.1224,
                       -0.1157, -0.1246,  0.0346,  0.1926, -0.0433,  0.0785,  0.2113,  0.0808,
                       -0.143

In [32]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[ 0.0169,  0.1424, -0.1518,  ...,  0.0480, -0.0597, -0.1075],
        [ 0.2097,  0.2059, -0.1069,  ...,  0.0249, -0.0938,  0.0879],
        [ 0.2004,  0.0726,  0.1240,  ...,  0.0409, -0.1526, -0.0526],
        ...,
        [ 0.1639, -0.0624, -0.0996,  ..., -0.1505,  0.1624,  0.0756],
        [-0.1191, -0.1694,  0.1929,  ..., -0.1619,  0.1548, -0.1295],
        [-0.1487,  0.2046,  0.0508,  ..., -0.2040,  0.1116, -0.1737]],
       requires_grad=True)
Parameter containing:
tensor([-0.0049, -0.1167,  0.2088, -0.0014,  0.1691,  0.2093,  0.1897,  0.0798,
         0.2109, -0.1295, -0.0762, -0.0910, -0.1965,  0.0270, -0.1057,  0.0457,
         0.1269, -0.1882, -0.0112,  0.0924,  0.0826,  0.1745, -0.1948, -0.1042,
         0.1448, -0.0194,  0.0637, -0.0856, -0.1919, -0.1870,  0.1220, -0.0128,
         0.0967,  0.0030,  0.1070,  0.1645, -0.0795, -0.1394,  0.1089,  0.1501,
         0.1717,  0.1272, -0.0787,  0.0428,  0.2186, -0.0477, -0.1626, -0.1337,
        -0.1966

In [36]:
class FixedHiddenMPL(nn.Module):
    def __init__(self):
        super().__init__()
        #不计算梯度的随机权重参数，在训练中保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        #使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        #复用全连接层。相当于两个全连接层共享参数
        X = self.linear(X)
        #控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [37]:
net = FixedHiddenMPL()
net(X)

tensor(0.0459, grad_fn=<SumBackward0>)

In [39]:
class NestMPL(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

In [40]:
chimera = nn.Sequential(NestMPL(), nn.Linear(16, 20), FixedHiddenMPL())
chimera(X)

tensor(0.0090, grad_fn=<SumBackward0>)